fastapi.exceptions.HTTPException: 500: Error fetching weather: 500: Error fetching weather data: API Error 429: {"reason":"Minutely API request limit exceeded. Please try again in one minute.","error":true}
(.venv) yeet@yeet:/mnt/Windows-SSD/Users/yvavi/yeet

In [10]:
import pandas as pd

# Load the CSV file
input_file = "input.csv"  # Replace with your input file name
output_file = "result.csv"  # Replace with your desired output file name

# Read the CSV file with proper handling of mixed types
df = pd.read_csv(input_file, dtype={"latitude": "str", "longitude": "str"}, low_memory=False)

# Drop rows where latitude or longitude is NA
df = df.dropna(subset=["latitude", "longitude"])

# Convert latitude and longitude to numeric, coercing invalid values to NaN
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

# Filter out rows with invalid latitude or longitude
invalid_lat_lon_df = df[~(df["latitude"].between(-90, 90)) | ~(df["longitude"].between(-180, 180))]
invalid_lat_lon_count = invalid_lat_lon_df.shape[0]
print(f"Number of rows with invalid latitude or longitude after removing NA: {invalid_lat_lon_count}")

# Remove invalid latitude/longitude rows from the main DataFrame
df = df[(df["latitude"].between(-90, 90)) & (df["longitude"].between(-180, 180))]

# Drop rows where 'pincode' or 'statename' is NA
df = df.dropna(subset=["pincode", "statename"])

# Group by 'statename' and calculate the average latitude and longitude, ignoring NA values
state_avg = df.groupby("statename", dropna=True)[["latitude", "longitude"]].mean().reset_index()

# Merge the average latitude and longitude back to the original DataFrame
result = df[["pincode", "statename"]].drop_duplicates()
result = result.merge(state_avg, on="statename", how="left")

# Save the result to a new CSV file
result.to_csv(output_file, index=False)

print(f"Result saved to {output_file}")

Number of rows with invalid latitude or longitude after removing NA: 225
Result saved to result.csv


In [11]:
# Read the result.csv file
import pandas as pd
output_file = "result.csv"  # Replace with your output file name
result_df = pd.read_csv(output_file)

# Extract unique combinations of latitude and longitude
unique_lat_lon = result_df[["latitude", "longitude"]].drop_duplicates()

# Save the unique combinations to a new CSV file
unique_lat_lon.to_csv("unique_lat_lon.csv", index=False)

print("Unique latitude and longitude combinations saved to unique_lat_lon.csv")

Unique latitude and longitude combinations saved to unique_lat_lon.csv


In [3]:
import pandas as pd

# Load the result.csv file
output_file = "unique_lat_lon.csv"  # Replace with your output file name
result_df = pd.read_csv(output_file)

# Round latitude and longitude to 6 decimal places to avoid floating-point precision issues
result_df["latitude"] = result_df["latitude"].round(6)
result_df["longitude"] = result_df["longitude"].round(6)

# Find duplicate rows based on latitude and longitude
duplicates = result_df.duplicated(subset=["latitude", "longitude"], keep=False)

# Count the number of duplicate rows
num_duplicates = duplicates.sum()

print(f"Number of duplicate rows based on latitude and longitude: {num_duplicates}")

Number of duplicate rows based on latitude and longitude: 0


In [12]:

df = pd.read_csv("unique_lat_lon.csv")
# Filter out rows with invalid latitude or longitude
invalid_lat_lon_df = df[~(df["latitude"].between(-90, 90)) | ~(df["longitude"].between(-180, 180))]
invalid_lat_lon_count = invalid_lat_lon_df.shape[0]
print(f"Number of rows with invalid latitude or longitude after removing NA: {invalid_lat_lon_count}")

Number of rows with invalid latitude or longitude after removing NA: 0


In [1]:
import pandas as pd
import json

# Load the result.csv file
input_file = "result.csv"  # Replace with your file path
df = pd.read_csv(input_file)

# Extract unique states
unique_states = df["statename"].dropna().unique().tolist()

# Save the unique states to a JSON file
output_file = "unique_states.json"
with open(output_file, "w") as json_file:
    json.dump({"states": unique_states}, json_file, indent=4)

print(f"Unique states saved to {output_file}")

Unique states saved to unique_states.json
